In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import polars as pl
import aiohttp
from tqdm.asyncio import tqdm

import compute_compliance
import validators

In [6]:
# load in the data
df = (pl
      .read_csv('paylesshealth/hospitals.csv')

    # some of the cdm_urls are like 
    # url1|url2 so we need to explode those out
    # into separate rows

      .with_column(
        pl.col('cdm_url').str.split('|'))
      .explode('cdm_url'))

In [14]:
async with aiohttp.ClientSession(raise_for_status = True) as session:
    for url in df['cdm_url'][14:]:
        print(url)
        if url is not None:
            if validators.url(url):
                await compute_compliance.checker(session, url)

https://southcoastglobalmedicalcenter.com/wp-content/uploads/2019/10/Pricing-Transparency-South-Coast-ChargeMaster-12-2019.xlsx
None
https://losrobleshospital.com/about/legal/pricing-transparency-cms-required-file-of-standard-charges.dot
https://search.hospitalpriceindex.com/hpi2/hospital/SouthernCaliforniaHospitalatCulverCity/7922or
https://www.shermanoakshospital.org/202546649_ShermanOaksHospital_standardcharges.json
None
https://www.montclair-hospital.org/202898771_MontclairHospitalMedicalCenter_standardcharges.json
https://healthy.kaiserpermanente.org/content/dam/kporg/final/documents/health-plan-documents/coverage-information/941105628-antioch-medical-center-standard-charges-ncal-en.xlsx
https://www.providence.org/external/050761_Providence-Cedars-Sinai-Tarzana-Medical-Center_StandardCharges.json
None
https://shastaregional.com/wp-content/uploads/2022/08/263487583_ShastaRegionalMedicalCenter_standardcharges.json
https://healthy.kaiserpermanente.org/content/dam/kporg/final/document

In [7]:
async with aiohttp.ClientSession(raise_for_status = True) as session:
        t = df.sample(10)
        cols = ['gross_chk', 'minmax_chk', 'cash_chk', 'generic_chk', 'insurer_chk', 'filename_chk', 'err']
        t[cols] = await tqdm.gather(*(compute_compliance.checker(session, url) for url in t['cdm_url']))

        # Add up all the values (except the error col) where the col equals 1
        t = t.with_column(
            pl.fold(pl.lit(0), lambda score, v: score + v.is_in([1]), cols[:-1]).alias('score')
        )

        t = t.sort('score')

here


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
t

ccn,name,state,city,homepage,cdm_indirect_url,cdm_url,gross_chk,minmax_chk,cash_chk,generic_chk,insurer_chk,filename_chk,err,score
str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64
"""260321""","""BEACON SURGERY...","""MO""","""LEES SUMMIT""","""https://beacon...",null,null,0,0,0,0,0,0,1,0
"""340168""","""WILMINGTON TRE...","""NC""","""WILMINGTON""","""https://www.wi...",null,null,0,0,0,0,0,0,1,0
"""052057""","""SONOMA SPECIAL...","""CA""","""SEBASTOPOL""","""https://sonoma...","""https://sonoma...","""https://apps.p...",0,0,0,0,0,0,0,0
"""244005""","""CHILD & ADOLES...","""MN""","""WILLMAR""",null,null,null,0,0,0,0,0,0,1,0
"""370039""","""HILLCREST HOSP...","""OK""","""CLAREMORE""","""https://hillcr...","""https://hillcr...","""https://coc.ar...",1,-1,-1,-1,1,-1,0,2
"""363306""","""DAYTON CHILDRE...","""OH""","""DAYTON""","""https://www.ch...","""https://www.ch...","""https://www.ch...",1,-1,1,1,1,-1,0,4
"""454110""","""OCEANS BEHAVIO...","""TX""","""MIDLAND""","""https://oceans...","""https://oceans...","""https://oceans...",1,1,1,1,1,-1,0,5
"""050127""","""WOODLAND MEMOR...","""CA""","""WOODLAND""","""https://www.di...","""https://www.di...","""https://www.di...",1,1,1,1,1,1,0,6
"""010023""","""BAPTIST MEDICA...","""AL""","""MONTGOMERY""","""https://www.ba...","""https://www.ba...","""https://res.cl...",1,1,1,1,1,1,0,6


In [9]:
t = await compute_compliance.check(df)

In [12]:
t

<coroutine object check at 0x166cd9af0>